Задание 1
На листе "Рецептура" файла себестоимостьА_в1.xlsx для области "Пшеничный хлеб" рассчитать 
себестоимость всех видов продукции.

In [3]:
#import xlwings as xw
import os
import pandas as pd
import numpy as np
import datetime

#ws=b value=values
a = xw.Book('себестоимостьА_в1.xlsx')
b = a.sheets[0]
table = b['рцп_пшеничный_хлеб'].options(empty=0)
value = table.value
value_price = [i for i in value if i[0] == 'Цена ресурсов, руб.'][0]
value_price = value_price[5:-4]
value_resours = value[3:7]
for i in range(len(value_resours)):
    value_resours[i] = value_resours[i][5:-4]

sum = []
for el in value_resours:
    sum1 = 0
    for i in range(len(el)):
        sum1 += el[i]*value_price[i]
    sum.append(sum)
sum

NameError: ignored

Задание 2
Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный 
хлеб"

In [4]:
b['P7'].options(transpose=True).value = sum

NameError: ignored

Задание 3
Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области

In [ ]:
b['P5'].value = 'Себестоимость единицы продукции'

Задание 1
Загрузите данные из файлов reviews_sample.csv (ЛР2) и recipes_sample.csv (ЛР5) в виде pd.DataFrame. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: id, name, minutes, submitted, description, n_ingredients

In [5]:
reviews = pd.read_csv('reviews_sample.csv')
reviews.rename(columns={reviews.columns[0] : 'index'}, inplace=True)
recipes = pd.read_csv('recipes_sample.csv')
recipes = recipes.drop('contributor_id', axis=1)

Задание 2
Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл recipes.xlsx. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц.

In [7]:
import random
import openpyxl
randoms = random.sample(range(0, len(recipes)), int(len(recipes)/20))
randoms1 = random.sample(range(0, len(reviews)), int(len(reviews)/20))

f = r'C:\\Users\\Влад\\Desktop\\учеба\\тод\\лр4\\data\\recipes.xlsx'
wb = openpyxl.Workbook()
wb.save(f)
recipes['submitted'] = pd.to_datetime(recipes['submitted'])
reviews['date'] = pd.to_datetime(reviews['date'])
file = xw.Book(f)
file.sheets[0].name = "Рецепты"
file.sheets.add(name="Отзывы", after=file.sheets[0].name)
b = file.sheets[0]
b1 = file.sheets[1]
lst = [recipes.loc[rand] for rand in randoms]
df = pd.DataFrame(lst)
b.activate()
b['A1'].options(index=False).value = df
b['D:D'][1:].number_format = 'ДД.ММ.ГГГГ'
b['E:E'][1:].number_format = '0'

b1.activate()
lst1 = [reviews.loc[rand] for rand in randoms1]
df1 = pd.DataFrame(lst1)
b1['A1'].options(index=False).value = df1
b1['E:E'][1:].number_format = '0'

NameError: ignored

Задание 3
Используя xlwings, добавьте на лист Рецепты столбец seconds_assign, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
seconds = [el*60 for el in df['minutes'].tolist()]
seconds = pd.DataFrame(seconds)
seconds.columns = ['seconds_assign']
b[f'I1:I{len(df)}'].options(index=False).value = seconds

Задание 4
Используя xlwings, добавьте на лист Рецепты столбец seconds_formula, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
b['J1'].value = 'seconds_formula'
b['J:J'][1:].formula = f'=C2 * 60'

Задание 5
Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
b['I1:J1'].font.bold = True
b['I:J'].api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

Задание 6
Раскрасьте ячейки столбца minutes в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
for el in b['C:C'][1:].expand('down'):
    if el.value < 5:
        el.color = (0, 255, 0)
    elif el.value > 10:
        el.color = (255, 0, 0)
    else:
        el.color = (255, 255, 0)

Задание 7
Добавьте на лист Рецепты столбец n_reviews, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
recipe_ids = []
for el in lst:   
    recipe_ids.append(el.id)  
n_reviews = reviews.groupby('recipe_id').count()['review']
recipe_n_rev = []
for el in recipe_ids:  
    try:
        recipe_n_rev.append(n_reviews[el])
    except KeyError:
        recipe_n_rev.append(0)
for i in range(len(recipe_n_rev)):
    recipe_n_rev[i] = [recipe_n_rev[i]]
b['K1'].value = 'n_reviews'
b['K:K'][1:].expand('down').value = recipe_n_rev 

Задание 8
 Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
def validate():
    recipe_id = b['B:B'][1:].value
    for i,j in enumerate(reviews.range('A2').expand().value):
        if j[2] not in recipe_id or not 0.0 <= j[4] <= 5.0:
            reviews.range(f'A{i+1}:F{i+1}').color = (255, 0, 0)

In [ ]:
validate()

Задание 9
В файле recipes_model.csv находится модель данных предметной области "рецепты". При помощи пакета csv считайте эти данные. При помощи пакета xlwings запишите данные на лист Модель книги recipes_model.xlsx, начиная с ячейки A2, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
import csv
with open('recipes_model.csv', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter = "\t")
    list_ = []
    for i in reader:
        list_.append(','.join(i).split(','))
print(list_)

In [ ]:
wb = xw.Book()
wb.sheets.add('Модуль')
b = wb.sheets['Модуль']
b.range('A2').value = list_[:]